# Лабораторная работа №3
## Выполнил студент группы БСТ2004 Зенин Михаил Витальевич

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [ ]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [ ]:
import time
def KnuthMorrisPratt(text, pattern):

    # индексирование образца
    pattern = list(pattern)

    # таблица сдвигов, изначально все сдвиги как в прямом поиске - 1
    shifts = [1] * (len(pattern) + 1)
    shift = 1
    for pos in range(len(pattern)):
        while shift <= pos and pattern[pos] != pattern[pos-shift]:
            shift += shifts[pos-shift] #cдвиг по максимальному префиксу увеличивается
        shifts[pos + 1] = shift

    # непосредственно поиск
    startPos = 0
    matchLen = 0
    for c in text:
        while matchLen == len(pattern) or matchLen >= 0 and pattern[matchLen] != c: # если дошли до длины образца или не нашли совпадение
            startPos += shifts[matchLen] #сдвигаем начальную позицию согласно таблице сдвигов
            matchLen -= shifts[matchLen]
        matchLen += 1
        if matchLen == len(pattern):
            print("Found pattern at index {}".format(startPos))

text = input()
pattern = input()
t = time.perf_counter()
KnuthMorrisPratt(text, pattern)
print("Алгоритм Кнута-Морриса-Пратта: {0:.6f} сек".format ((time.perf_counter() - t)))

Seventy-seven benevolent elephants
ev
Found pattern at index 1
Found pattern at index 9
Found pattern at index 17
Алгоритм Кнута-Морриса-Пратта: 0.001849 сек


#### Упрощенный алгоритм Бойера-Мура

In [ ]:
def BoyerMoore(text, pattern):
    lst = list(pattern)
    sizePattern = len(lst)
    dct = {}
    for i in reversed(range(sizePattern-1)):
        if dct.get(lst[i]) == None:
            dct [lst[i]] = sizePattern - i - 1 #индексация с конца до начала, не считая последнего символа. вносится в словарь
    last = lst[sizePattern - 1]
    if dct.get(last) == None:
        dct [last] = len(lst) # Если последний символ образа уникальный для всего образа, то ему назначается смещение на всю длину образа
    dct['*'] = sizePattern #Разделитель, не присутствующий в тексте. На него назначается смещение на всю длину образа

    size = len(text)
    if size >= sizePattern:
        head = sizePattern - 1       # счетчик проверяемого символа в строке

        while(head < size): #пока указатель не сдвинется за пределы текста
            k = 0
            j = 0
            flBreak = False
            for j in range(sizePattern - 1, -1, -1): #отсчет с конца образа с шагом -1
                if text[head - k] != lst[j]:
                    if j == sizePattern - 1: #если несовпадающий символ последний в образе
                        off = dct[text[head]] if dct.get(text[head], False) else dct['*']  # смещение, если не равен последний символ образа
                    else:
                        off = dct[lst[j]]   # смещение, если не совпадающий символ не последний в образе

                    head += off    # смещение счетчика строки
                    flBreak = True  # если несовпадение символа, то flBreak = True
                    break

                k += 1          # смещение для сравниваемого символа в строке text[head - k], сдвиг влево

            if not flBreak:          # если дошли до начала образа, значит, все его символы совпали
                print(f"Pattern found at index {head - k + 1}")
                break
        else:
            print("Pattern not found")
    else:
        print("Pattern not found")

text = input()
pattern = input()
t = time.perf_counter()
BoyerMoore(text, pattern)
print("Алгоритм Бойера-Мура-Хорспула: {0:.6f} сек".format ((time.perf_counter() - t)))
t = time.perf_counter()
text.find(pattern)
print("Встроенный алгоритм поиска подстроки: {0:.6f} сек".format ((time.perf_counter() - t)))

Seventy-seven benevolent elephants
ev
Pattern found at index 1
Алгоритм Бойера-Мура-Хорспула: 0.000754 сек
Встроенный алгоритм поиска подстроки: 0.000114 сек


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [ ]:
# Функция считает количество инверсий,
# то есть количество случаев, когда некое
# a > b , но при этом расположено раньше в переданном массиве
N = 4
def getInvCount(arr):
    inv_count = 0
    for i in range(N * N - 1):
        for j in range(i + 1,N * N):
            # если (arr[i], arr[j]) такое, что
            # i < j и arr[i] > arr[j]
            if (arr[j] and arr[i] and arr[i] > arr[j]):
                inv_count += 1
         
    return inv_count
 
 
# находим в каком ряду (от нижнего) находится пустое поле
def findXPosition(puzzle):
    for i in range (N*N - 1, -1, -1):
        if puzzle[i] == 0:
            if i < 4:
                return 4
            if i > 3 and i < 8:
                return 3
            if i > 7 and i < 12:
                return 2
            else:
                return 1
 
 
# Возвращает true, если такую расстановку пятнашек можно решить
def isSolvable(puzzle):
    # подсчитываем количество инверсий
    invCount = getInvCount(puzzle)
 
    # если размерность введенного поля нечетная, то возвращаем true
    # если количество инверсий четное
    if (N & 1):
        return ~(invCount & 1)
 
    else:    # если размерность четная
        pos = findXPosition(puzzle)
        if (pos & 1): # если пустое поле стоит в нечетном ряду (снизу отсчет)
            return ~(invCount & 1) #то возвращаем истину, если инверсий четное кличество
        else:
            return invCount & 1 #то возвращаем истину, если инверсий нечетное кличество
     
 
##print("Solvable") if  isSolvable(puzzle1) else print("Not Solvable")
puzzle1 = [ # решаемый
12, 1, 10, 2,
7, 11, 4, 14,
5, 0, 9, 15, # 0 означает пустое место
8, 13, 6, 3]

puzzle = [3, 9, 1, 15, #нерешаемый
    14, 11, 4, 6,
    13, 0, 10, 12,
    2, 7, 8, 5]
res = isSolvable(puzzle)
if res:
    print([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0])
else:
    print([])

[]


### Вывод